# 第7章 因果推論のための時系列解析

## 7.1 時系列解析を用いた因果推論の全体像

### 7.1.1 時系列解析と因果推論の関係

これまで結果変数や共変量、交絡因子に対して時間依存がないという仮定を置いていたが、時間依存性を考慮する因果推論も現実問題として存在する。

![図7.2](images/chap7/image.png)

時間変化を伴う因果効果の推定

* 処置が単体の場合: 特定の1つのイベントの効果を推定する(処置の前後の効果を検証)
  1. 処置群と対照群がそれぞれ存在する場合: DiD、SCを用いる事ができる
  2. 処置群しか存在しない場合: 時系列解析を用いる

     * イベントの効果検証: 実際の売上データに対して、イベント前の売上データを時系列モデルで学習。イベント以降の予測値を反実仮想として算出。予実差を因果効果とする。
* 処置が複数の場合:
  * 複数のメディアの効果検証: マーケティング・ミックスモデリング(Marketing Mix Modeling: MMM)を使用できる

### 7.1.2 時系列解析の活用

* 時系列解析
  * 予測: 売上や需要の将来予測など。予測精度が優先され解釈性が二の次になる
  * 因果推論: 時系列を伴う効果検証など。ここでは処置群のみが存在する場合において、単体施策と複数施策について考える
    * 単体施策: 対照群のデータを予測し、その予測値と実測値の差を因果効果とする
    * 複数施策: 回帰分析と同様に各施策の効果を推定し比較する

実務に活用するにあたって、特にマーケティングにおけるPDCAサイクルがある。

* Plan: 売上・需要予測(数理最適化を組み合わせル事例もある: 配車サービスDiDi)
* Do: レコメンドなどの準備された施策を実行
* Check: 因果推論などを用いて効果検証を行う
* Action: 今後の改善

## 7.2 時系列解析の全体像

### 7.2.1 時系列解析の概要

扱うデータの具体例: 経済データ(株価、為替など)、環境データ(気温、降水量など)

データサイエンスの科学的な貢献に以下の3つがある

* 記述(Description): この項
* 予測(prediction): [7.2.2~7.2.4項](#722-時系列解析の実行手順)
* 因果推論(counterfactual prediction): [7.3項以降](#73-時系列解析の因果推論への適用)

これに沿って時系列解析を説明する。

#### 記述

時系列解析で可視化・抽出すべき情報

1. 季節性/周期性: 一定の時間間隔で繰り返し現れるパターン

   年、月、週などの単位。花粉、四季
2. トレンド: 長期にわたり増加または減少する大まかな方向性
3. 外因性: イベント、祝日などの外部の要因が及ぼす性質

   例えば経済では、国内外の情勢や経済情景、天候、災害など
4. 自己相関: 異なる時点間の相関

   ある時系列データの特定の時間における値が、その前後の時間の値とどの程度相関があるかを示す統計量。

   自己相関関数(事項相関の度合いを表す)

   * $k$: 時間差
   * $R_k$: 時間差kのときの自己相関係数
   * $Cov(Y_t, Y_{t-k})$: 自己共分散
   * $Var(Y_t), Var(Y_{t-k})$: 分散

   $$
   R_k
   = \frac{Cov(Y_t, Y_{t-k})}
          {\sqrt{Var(Y_t)Var(Y_{t-k})}}
   $$

   自己相関係数は-1から1までの値を取る
5. ノイズ: 取り出したい情報以外の不要な情報

   なんの情報を取り出したいかによって、ノイズの具体的に定義は変わる。特に時系列解析ではホワイトノイズがよく使用される

   * ホワイトノイズ: 期待値0、分散が一定一定の自己相関を持たないノイズ

* 定常性:
  
  同時分布や同時分布や基本統計量の時間不変性に関するもの。
  
  何を不変とするかによって、弱定常性と強定常性に分類される
  * 弱定常性: 時系列データの以下のパラメータが時間差(ラグ)のみに依存する性質
    * 平均(期待値): $E[Y_t] = E[Y_{t-k}]$
    * 分散: $Var(Y_t) = Var(Y_{t-k})$
    * 共分散: $Cov(Y_t, Y_s) = Cov(Y_{t-k}, Y_{s-k})$
    
  * 強定常性: 弱定常性の3つのパラメータに加え、同時分布が時間差により変わらない性質
    * 同時分布: $p(Y_{t_1}, Y_{t_2}, \cdots Y_{t_n}) = p(Y_{t_1 + k}, Y_{t_2 + k}, \cdots Y_{t_n + k})$

    ![図7.10](images/chap7/image-1.png)

  最も基本的な強定常過程の例: 独立同一分布(independently and identically distributed: i.i.d.)系列。以下を満たす。

  各時点のデータが、
  1. 互いに独立
  2. かつ、同一の確率分布から生成される

* 非定常性: 弱定常性にも該当しないもの

定常性、非定常性は検定で確認可能
* Augmented Dickey Fuller(ADF)検定
* Kwiatkowski-Phillips-Schmidt-Shin(KPSS)検定
など

定常性を仮定することのメリット
* モデルのパラメータ推定が容易
* 定常性を持つ時系列は、未来の挙動の予測精度を高められる可能性が高い

非定常時系列データの扱いかた
* 簡易的な方法として時系列の差分を取るなど処理を行い、定常時系列に変換する方法
* 一方で非定常時系列をそのまま扱い、予測モデルを構築する方法もある

### 7.2.2 時系列解析の実行手順

#### 1. データ準備
十分な期間のデータが必要(季節性であれば数年分など)。だが、期間が長過ぎても良くない。例えばCovid-19などある時点で状況がガラッとかわっている場合などは予測精度が下がる場合がある。何にせよ背景知識が大事。

#### 2. 前処理、3. 学習・検証
第4章で解説した機械学習と同様(データ分割、学習、テスト)。

この際に可能であればクロスバリデーションとして、複数の学習期間とテスト期間を設定して精度検証を行う。時系列データでは過学習の検証時に複数の期間を検証する必要がある。

#### 4. 将来予測

必要な場合、学習モデルを用いて将来の予測値を算出。このとき、何時点先までを算出するかを指定する。

### 7.2.3 時系列モデルの種類

1. 分解可能なモデル
   * 構造時系列モデル: 
2. 分解不可能なモデル